In [1]:
import pandas as pd
import re 
import numpy as np
import os
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
from nltk.tokenize import word_tokenize
nltk.download("punkt")
from snowballstemmer import TurkishStemmer
from tqdm import tqdm
from datetime import datetime, timedelta

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\emine\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\emine\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Read data from source

Going from the Machine Learning directory to the Web Scraping directory.

In [3]:
def get_dataset_path(sub_directory,file_name):
    base_dir = os.getcwd()
    target_dir = os.path.dirname(base_dir)
    file_path = os.path.join(target_dir, sub_directory,file_name)
    return file_path

In [3]:
def read_data(sub_directory,file_name):
    file_path = get_dataset_path(sub_directory,file_name)
    df = pd.read_csv(file_path)
    return df

Converting all data to lowercase helps in the process of preprocessing and in later stages in the NLP application, when doing parsing.

In [4]:
def convert_to_lowercase(df):
    return df["review"].apply(lambda x:" ".join(x.lower() for x in x.split()))

This function is designed to detect and remove rare words from the dataset. However, it was not used due to its slow performance.

In [ ]:
def remove_rare_words(df, threshold):
    words = pd.Series(" ".join(df["review"]).split()).value_counts()
    rare_words = words[words <= threshold].index.tolist()
    return df["review"].apply(lambda x:" ".join(x for x in x.split() if x not in rare_words)) 
 

Removing punctuation can help simplify the text and improve the accuracy of NLP models.

In [6]:
def remove_punctuation(df):
    return df["review"].apply(lambda x : re.sub(r'[^\w\s]', '',x))       

Urls do not provide any sentiment information.

In [7]:
def remove_urls(df):
    return df["review"].apply(lambda x : re.sub(r"http\S+", "",x))

A single comment may appear multiple times on the 'DonanımHaber' website

In [8]:
def remove_duplicate_review(df):
    print('REMOVING DUPLICATE REVIEW....')
    return  df.drop_duplicates(subset = ["review"] ,keep='first', inplace=False)

`quoteOrjinalden alıntı` from the beginning of the quoted comments will be removed from the original.

In [9]:
def remove_quote(df):
    for index, row in df.iterrows():
        comment = row['review']
        cleaned_comment = re.sub(r'quoteOrjinalden alıntı\s+\w+', '', comment)
        df.at[index, 'review'] = cleaned_comment.strip()
    return df

In NLP, numbers are important in areas such as NER or POS, but numbers are not important for sentiment analysis.

In [10]:
def remove_numbers(df):
    return df["review"].apply(lambda x : re.sub("\S*\d\S*", "", x))

Special characters can cause problems, especially when transferring data to another platform. For example, Turkish characters cause problems when exported to Excel.

In [11]:
def remove_special_characters(df):
    return df["review"].apply(lambda x : re.sub('[^A-Za-z0-9]+', ' ', x))

Tokenization is the division of raw data into smaller, understandable tokens.This function divides comments in sentences into words.As we are using a pre-trained model, we will perform this process at later stages using the tokenization techniques specific to the model we are utilizing.

In [12]:
def tokenize_reviews(df):
    return df['review'].apply(lambda text: word_tokenize(text, language='turkish'))

Stopword removal helps reduce the size of text data and makes it easier to analyze.
Here are some common Turkish stopwords:"ve","ile","bir" ,"da", "de","bu"...

In [13]:
def remove_stopwords(df):
    sw = set(stopwords.words("turkish"))
#     return df["review"].apply(lambda tokens: [token for token in tokens if token not in sw])
    return df["review"].apply(lambda x:" ".join(x for x in x.split() if x not in sw))

In [14]:
# def remove_stopwords2(df):
#     sw = set(stopwords.words("turkish"))
#     return df["review"].apply(lambda tokens: print(tokens))

The purpose of lemmatization is to transform inflected words into their root form to simplify analysis and improve the accuracy of text processing tasks.But, lemmatization was not performed in the project due to concerns about its slow performance.

In [15]:
def lemmatize(df):
    turkStem = TurkishStemmer()
    lemmatized_reviews = []
    for review in tqdm(df['review'].values):
        review = ' '.join(turkStem.stemWord(word) for word in review.split())
        lemmatized_reviews.append(review.strip())
    df['review']=lemmatized_reviews
    return df

## Read data from source

### Dataset

Read training and testing data

In [51]:
df = read_data("WebScraping/Dataset","dataset.csv")
df_copy=df.copy(deep=True)
df.head()

,review
0,Bayiler satar artık 1.200 000 e burası türkiye...
1,Siz bu fiyatlara 200 bin ekleyin. Bayide bulam...
2,Bu motor bu kasaya daha çok yakışmış ...
3,850Bin TL eksik aracı almam için. Neyse 1 sene...
4,Eskiden milyoner olmak vardı derlerdi. Nerden ...


In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93991 entries, 0 to 93990
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  93991 non-null  object
dtypes: object(1)
memory usage: 734.4+ KB


In [53]:
print('-----SHAPE OF THE DATASET-----')
df.shape

-----SHAPE OF THE DATASET-----


(93991, 1)

## Brands

Read analyse data

In [16]:
file_path = get_dataset_path("WebScraping","Brands")
file_list = os.listdir(file_path)

brands = {}

for file in file_list:
     if file.endswith('.csv'):
        df_path = os.path.join(file_path, file)
        df_brand = pd.read_csv(df_path)
        brands[file[:-4]] = df_brand

In [17]:
brands["Opel"].head()

,review,date
0,Clio ile rekabet etmesi zor görünüyor ...,6 gün
1,Önü ve arkası bambaşka tellerden çalıyor. Maky...,6 gün
2,Bu elektrikli araçlara Menzil tarikatının birş...,6 gün
3,Ön farları mahvetmişler. Önceki hali çok daha ...,6 gün
4,208 ne zaman gelecek acaba. 100 kw kötü olmuş...,6 gün


## Dataset Preprocessing

In [54]:
print("We have {} NaN/Null values in dataset ".format(df.isnull().values.sum()))
print("Number of duplicate in  dataset : {} ".format(sum(df['review'].duplicated())))

We have 0 NaN/Null values in dataset 
Number of duplicate in  dataset : 2661 


Random comments were examined  to identify issues 

In [169]:
# random reviews
pd.set_option('display.max_colwidth', None)
print(df["review"].values[1])
print(df["review"].values[41])
print(df["review"].values[44])
print(df["review"].values[52])
print(df["review"].values[48])
pd.reset_option('display.max_colwidth')

siz bu fiyatlara 200 bin ekleyin. bayide bulamayacaksınız, galeri de üstüne karını koyup satacak. yeni düzen böyle.
şükredin. elhamdulillah 🙏
hyundai̇ fi̇yat li̇stesi̇< resime gitmek için tıklayın >sağ üstte azami /maksimum satış fiyatları yazıyor.< resime gitmek için tıklayın >
quote:orjinalden alıntı: hasanyy ben renault yetkili servis çalışanıyım.! 6 ay önce scenıc ıı aldım ve yıllık izne çıktım. samsun' dan fethiye ye gittim. tek şöför inanın hiç yorulmadım! çok sessiz ve rahat bir araç !! yüksek bir araç olduğu için içinde sıkışık ve basık hissi duymuyorsunuz. aile arabası diyorlar, evet ama biz iki kişiyiz uzun yol konforu için scenic ii aracını tercih ettim. kesinlikle öneririm. !!! araç jip gibi duruyor ama vergisi standart sade bir araç gibi alın başka araç kullandığınızda hemen inmek isteyeceksiniz katiliyorum, scenic ile ilgilenmeden once focus ya da c-max dusunuyordum. ne zaman ki scenic'in icine oturdum, digerleri bana oyuncak araba gibi geldi :)
mercedes-benz birkaç gün ö

In [170]:
# TO-DO
# remove duplicate review  ->OK
# remove punctuations      ->OK
# remove tags              ->OK   
# remove quote             ->OK
# remove numbers           ->OK
# remove urls              ->OK
# remove special characters->OK
# convert lowercase        ->OK
# remove rare words        ->çok yavaş çalışıyor

In [171]:
print("Number of review in  dataset : {} ".format(df.shape))
df = remove_duplicate_review(df)
print("Number of review in  dataset : {} ".format(df.shape))

Number of review in  dataset : (93991, 1) 
REMOVING DUPLICATE REVIEW....
Number of review in  dataset : (91105, 1) 


In [172]:
df["review"] = remove_punctuation(df)
df.head()

,review
0,bayiler satar artık 1200 000 e burası türkiye
1,siz bu fiyatlara 200 bin ekleyin bayide bulama...
2,bu motor bu kasaya daha çok yakışmış
3,850bin tl eksik aracı almam için neyse 1 sene ...
4,eskiden milyoner olmak vardı derlerdi nerden n...


In [173]:
print(df["review"].values[44])

hyundai fiyat listesi resime gitmek için tıklayın sağ üstte azami maksimum satış fiyatları yazıyor resime gitmek için tıklayın 


In [174]:
# rare_words = remove_rare_words(df,threshold)  

In [175]:
df["review"] = remove_urls(df)
print(df["review"].values[48])

mercedesbenz birkaç gün önce eserisi sedanlar için yeni bilgieğlence sistemini üç bölmeli dev ekran panellerle birlikte güncellemişti mercedes şimdi ise cisco ile ortaklığa gideren webex toplantılarını otomobillerine getiriyormercedesbenz eserisi ile toplantılara katılabilirsiniz2024 mercedesbenz eserisi wifi ve hücresel veri bağlantısı ile donatılacak yani sürücüler webex uygulamasını cep telefonuna ihtiyaç duymadan doğrudan aracın bilgieğlence sisteminin dokunmatik ekranında görünecek şekilde indirebilecek güvenlik açısından araç hareket halindeyken sadece sesli bağlantıya izin verilecek ancak park haline geçildiğinde görüntülü toplantılar içerik paylaşımı gibi özellikler de kullanılabilir olacakmobil dünya kongresi mwc kapsamında duyurulan bu ortaklık sayesinde yeni eserisi otomobiller mobil ofis alanlarına dönüşecek gibi görünüyor mercedes araçlarda üstün bir gürültü engelleme özelliğinin de olacağını söylerken bazı kullanım senaryoları da paylaştı mercedese göre özellikle mimarlar

In [176]:
print(df["review"].values[52])

quoteorjinalden alıntı hasanyy ben renault yetkili servis çalışanıyım 6 ay önce scenıc ıı aldım ve yıllık izne çıktım samsun dan fethiye ye gittim tek şöför inanın hiç yorulmadım çok sessiz ve rahat bir araç  yüksek bir araç olduğu için içinde sıkışık ve basık hissi duymuyorsunuz aile arabası diyorlar evet ama biz iki kişiyiz uzun yol konforu için scenic ii aracını tercih ettim kesinlikle öneririm  araç jip gibi duruyor ama vergisi standart sade bir araç gibi alın başka araç kullandığınızda hemen inmek isteyeceksiniz katiliyorum scenic ile ilgilenmeden once focus ya da cmax dusunuyordum ne zaman ki scenicin icine oturdum digerleri bana oyuncak araba gibi geldi 


In [177]:
df = remove_quote(df)
print(df["review"].values[52])

quoteorjinalden alıntı hasanyy ben renault yetkili servis çalışanıyım 6 ay önce scenıc ıı aldım ve yıllık izne çıktım samsun dan fethiye ye gittim tek şöför inanın hiç yorulmadım çok sessiz ve rahat bir araç  yüksek bir araç olduğu için içinde sıkışık ve basık hissi duymuyorsunuz aile arabası diyorlar evet ama biz iki kişiyiz uzun yol konforu için scenic ii aracını tercih ettim kesinlikle öneririm  araç jip gibi duruyor ama vergisi standart sade bir araç gibi alın başka araç kullandığınızda hemen inmek isteyeceksiniz katiliyorum scenic ile ilgilenmeden once focus ya da cmax dusunuyordum ne zaman ki scenicin icine oturdum digerleri bana oyuncak araba gibi geldi


In [178]:
df["review"] = remove_numbers(df)
df.head()

,review
0,bayiler satar artık e burası türkiye
1,siz bu fiyatlara bin ekleyin bayide bulamayac...
2,bu motor bu kasaya daha çok yakışmış
3,tl eksik aracı almam için neyse sene bekleye...
4,eskiden milyoner olmak vardı derlerdi nerden n...


In [179]:
# df["review"] = remove_special_characters(df)
# df.head()

In [180]:
df["review"] = remove_stopwords(df)
df.head()

,review
0,bayiler satar artık e burası türkiye
1,fiyatlara bin ekleyin bayide bulamayacaksınız ...
2,motor kasaya yakışmış
3,tl eksik aracı almam neyse sene bekleyem alırım
4,eskiden milyoner olmak vardı derlerdi nerden g...


In [182]:
df_copy.head()

,review
0,Bayiler satar artık 1.200 000 e burası türkiye...
1,Siz bu fiyatlara 200 bin ekleyin. Bayide bulam...
2,Bu motor bu kasaya daha çok yakışmış ...
3,850Bin TL eksik aracı almam için. Neyse 1 sene...
4,Eskiden milyoner olmak vardı derlerdi. Nerden ...


In [184]:
df["review"] = lemmatize(df)
df.head()

,review
0,bayi satar ar e buras türki
1,fiyat bin ekley bayi bulamayacak ga üst kar ko...
2,motor kasa yakış
3,tl eksik araç alma ne se bekleye alır
4,eski milyoner olmak var der ner gel bir ülke p...


In [185]:
df.head(10)

,review
0,bayi satar ar e buras türki
1,fiyat bin ekley bayi bulamayacak ga üst kar ko...
2,motor kasa yakış
3,tl eksik araç alma ne se bekleye alır
4,eski milyoner olmak var der ner gel bir ülke p...
5,arkadaş eytli arabas deyip takıldık corolla ge...
6,eski türkiye bayi git mi araç satmak bayi tak ...
7,fiyat listes yer stok listes paylaşıl faydal h...
8,yok sen on gecicentoyota site gordugu fiya ali...
9,alinmaz araba fiyat kadar zam gelecek sekil insaf


In [186]:
df_copy.head(10)

,review
0,Bayiler satar artık 1.200 000 e burası türkiye...
1,Siz bu fiyatlara 200 bin ekleyin. Bayide bulam...
2,Bu motor bu kasaya daha çok yakışmış ...
3,850Bin TL eksik aracı almam için. Neyse 1 sene...
4,Eskiden milyoner olmak vardı derlerdi. Nerden ...
5,Arkadaşlarla EYT'li arabası deyip takıldığımız...
6,Eski türkiyede bayiye gittin mi araç satmak iç...
7,Fiyat listesi yerine stok listesi paylaşılsa d...
8,Yok ya sen onu gecicenToyotanin sitesinde gord...
9,"Alinmaz bu arabalar bu fiyatlara, daha ne kada..."


In [46]:
def preprocessing1(df):
    print("Number of review in  dataset : {} ".format(df.shape))
    df = remove_duplicate_review(df)
    print("Number of review in  dataset : {} ".format(df.shape))
    df["review"] = remove_punctuation(df)
    df["review"] = remove_urls(df)
    df = remove_quote(df)
    df["review"] = remove_numbers(df)
    df["review"] = convert_to_lowercase(df)
    df["review"] = remove_stopwords(df
#     df["review"] = lemmatize(df)
#     df["review"] = tokenize_reviews(df)
    df.to_csv('preprocessing_dataset.csv')
    return df

Reusing comments instead of repeatedly going through the dataset improved performance.

In [20]:
def preprocessing(df):
    preprocessed_reviews = []
    sw = set(stopwords.words("turkish"))
    for review in tqdm(df['review'].values):
        review = re.sub(r'[^\w\s]', '', review)
        review = re.sub(r'http\S+', '', review)
        review = re.sub(r'quoteOrjinalden alıntı\s+\w+', '', review)
        review = re.sub(r'\S*\d\S*', '', review)
        review = re.sub(r'\S*\d\S*', '', review)
        review = ' '.join(word.lower() for word in review.split() if word.lower() not in sw)
        preprocessed_reviews.append(review.strip())
    df['review']=preprocessed_reviews
    return df

DonanımHaber website contain time information in various formats such as "day," "1 week," "2 days," and "now," and that this time information is being converted to a date format.

In [22]:
def time_convert(df):
    converted_dates = []
    for date in tqdm(df['date'].values):
        
        if len(date.split()) >= 2:
            num, unit = date.split()[0], date.split()[1]
            if num == "geçen":
                num = 1

            if unit == "gün":
                time = (datetime.now() - timedelta(days=int(num))).strftime('%d.%m.%Y')
            elif unit =="hafta":
                time = (datetime.now() - timedelta(weeks=int(num))).strftime('%d.%m.%Y')
            elif unit =="ay":
                time = (datetime.now() - timedelta(weeks=int(num))).strftime('%d.%m.%Y')
#             az önce,... saat,... dk
            else:
                time = datetime.now().strftime('%d.%m.%Y')
#         dün,şimdi,bugün
        else:
            if "dün" in date:
                time = (datetime.now() - timedelta(days=1)).strftime('%d.%m.%Y')
            elif "şimdi" in date or "bugün" in date:
                time = datetime.now().strftime('%d.%m.%Y')              
            else:
                time = date
        
        converted_dates.append(time)
    df['date']=converted_dates
    return df

Preprocess and save

In [55]:
preprocessed_df = preprocessing(df)
preprocessed_df.to_csv('preprocessed_dataset.csv')

100%|█████████████████████████████████████████████████████████████████████████| 93991/93991 [00:08<00:00, 10599.16it/s]


In [23]:
for name, df in brands.items():
    df = remove_duplicate_review(df)
    df = preprocessing(df)
    brands[name] = df
    df = time_convert(df)
    
    if not os.path.exists("Brands"):
        os.makedirs("Brands")
    df.to_csv(os.path.join("Brands", f'{name}.csv'), index=False)

REMOVING DUPLICATE REVIEW....


100%|█████████████████████████████████████████████████████████████████████████████| 369/369 [00:00<00:00, 10504.19it/s]
C:\Users\emine\AppData\Local\Temp\ipykernel_21920\3736048588.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review']=preprocessed_reviews
100%|████████████████████████████████████████████████████████████████████████████| 369/369 [00:00<00:00, 123342.22it/s]
C:\Users\emine\AppData\Local\Temp\ipykernel_21920\2096692961.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date']

REMOVING DUPLICATE REVIEW....


100%|████████████████████████████████████████████████████████████████████████████| 957/957 [00:00<00:00, 159157.37it/s]


REMOVING DUPLICATE REVIEW....


...

100%|████████████████████████████████████████████████████████████████████████████| 613/613 [00:00<00:00, 123048.98it/s]


REMOVING DUPLICATE REVIEW....


100%|████████████████████████████████████████████████████████████████████████████| 115/115 [00:00<00:00, 114462.50it/s]


REMOVING DUPLICATE REVIEW....


100%|██████████████████████████████████████████████████████████████████████████| 2341/2341 [00:00<00:00, 138079.96it/s]


In [105]:
brands["Opel"].head()

,review,date
0,clio rekabet etmesi zor görünüyor,02.10.2023
1,önü arkası bambaşka tellerden çalıyor makyajsı...,02.10.2023
2,elektrikli araçlara menzil tarikatının birşeyl...,02.10.2023
3,ön farları mahvetmişler önceki hali iyiydi,02.10.2023
4,zaman gelecek kw kötü olmuş kw varken,02.10.2023
